In [ ]:
import nltk
import sqlite3
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [129]:
# Data Loading
# dbs must be in same folder as notebook

DATABASE1_Name = 'engA.db'
DATABASE2_Name = 'engE.db'
sidLow = 0
sidHigh = 9999999
def load_data_into_pandas(targetDatabase):
    print("Loading {0}".format(targetDatabase))
    con = sqlite3.connect(targetDatabase)
    db = pd.read_sql_query("""SELECT sid, cid, clemma, tag, tags FROM concept WHERE sid >= {} AND sid <= {};""".format(str(sidLow), str(sidHigh)),con)
    print("{0} Loaded".format(targetDatabase))

    return db


database_1 = load_data_into_pandas(DATABASE1_Name)
database_2 = load_data_into_pandas(DATABASE2_Name)



Loading engA.db
engA.db Loaded
Loading engE.db
engE.db Loaded


In [130]:
# Data Extraction and helper Functions

def get_unique_words_from_db(db):
    allWords = db.drop(['tag','sid', 'cid','tags'], axis = 1)
    allWords = allWords.drop_duplicates()
    return allWords

def innerJoin( db_a, db_b):
    return db_a.merge(db_b, how='inner', on=['sid', 'cid'] ,suffixes=['_db_a','_db_b'], sort=False, validate = 'one_to_one' )
     
def getCountofEachWord(db):
    allWordCounts = db.groupby(['clemma']).size()
    return  allWordCounts

def checkAgreement(db):
    def agreementHelper(row):
        return row['tag_db_a'] == row['tag_db_b']
    agreements = db.apply(agreementHelper, axis =1 )
    result = db.copy()
    result['agreement'] = agreements
    return result
    

In [131]:
# Generating List of Unique Words
database_1_unique_words = get_unique_words_from_db(database_1)
database_2_unique_words = get_unique_words_from_db(database_2)

# Generating List of word counts
database_1_words_counts = getCountofEachWord(database_1)
database_2_words_counts = getCountofEachWord(database_2)

# Joining the Two DBs
combined_database = innerJoin(database_1, database_2)

# Checking Agreement
combined_database_with_agreement = checkAgreement(combined_database)


In [132]:
def add_column_by_inner_join(db,add):
    return db.merge(add, how='left', on=['clemma_db_a'], sort=False, validate = 'one_to_one' ).fillna(0)

# Grouping 
# agreementGrouping
combined_database_with_agreement_grouped_count = combined_database_with_agreement.groupby(['clemma_db_a','agreement']).size().to_frame().rename(columns = {0:'agreementCount',}).reset_index()
combined_database_with_agreement_grouped_count_positives = combined_database_with_agreement_grouped_count[combined_database_with_agreement_grouped_count['agreement'] == True ].drop(['agreement'], axis = 1)
# combined_database_with_agreement_grouped_count.columns = ['agreementCount']

#wordCount Grouping
combined_database_with_agreement_wordCount = combined_database_with_agreement.groupby(['clemma_db_a']).size().to_frame().rename(columns = {0:'wordCount',}).reset_index()
# combined_database_with_agreement_wordCount.columns = ['wordCount']

combined_agreement_database = add_column_by_inner_join(
    combined_database_with_agreement_wordCount,
    combined_database_with_agreement_grouped_count_positives)




In [133]:

def calculatePercentageAgreement(agreement_db):
    def percentageHelper(row):
        # do we care about spaces?????? use
        # return row['tag_db_a'].strip() == row['tag_db_b'].strip()
        return  (row['agreementCount']/row['wordCount']) * 100

    percentages = agreement_db.apply(percentageHelper, axis =1 )
    result = agreement_db.copy()
    result['Percent'] = percentages
    return result

# Change ascending to flip
combined_agreement_database_withPercents = calculatePercentageAgreement(combined_agreement_database).sort_values(['Percent'], ascending = True )

In [134]:
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 0)
combined_agreement_database_withPercents.head(9999)

,clemma_db_a,wordCount,agreementCount,Percent
4296,excuse me,2,1,50.000000
4295,excuse,8,7,87.500000
7698,me,184,183,99.456522
0,!,1,1,100.000000
9187,perfumed,1,1,100.000000
9188,perfunctory,1,1,100.000000
9189,perhaps,19,19,100.000000
9190,period,36,36,100.000000
9191,period of time,2,2,100.000000
...,...,...,...,...


In [135]:
# To find list of words in db
def findWordsin_agreement_database(db,list_of_clemma):
    return db.loc[db['clemma_db_a'].isin(list_of_clemma)]

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 0)

a = findWordsin_agreement_database(combined_agreement_database_withPercents, ['dessert'])
a.head(20)

,clemma_db_a,wordCount,agreementCount,Percent
3438,dessert,26,26,100.0


In [136]:
# Excel conversion
combined_agreement_database_withPercents.to_excel("output2.xlsx")